<a href="https://colab.research.google.com/github/kuzmatsukanov/kuzma_omri_noa_data_project/blob/main/H_M_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminary loadings

In [1]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import load_model
import tensorflow as tf
from sklearn.metrics.pairwise import cosine_similarity
import os
import ipywidgets as widgets

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Load the trained model of feature recognitions
model = load_model('/content/drive/MyDrive/ds_project_dataset/my_model.h5')

# Lists of the class labels
colour_lst = \
    ['Beige', 'Black', 'Blue', 'Brown',
       'Green', 'Grey', 'Metal', 'Mole',
       'Orange', 'colour_Pink', 'Purple', 'Red',
       'Turquoise', 'White', 'Yellow']

product_lst = \
    ['Bags', 'Garment Full body',
       'Garment Lower body', 'Garment Upper body',
       'Shoes', 'Socks & Tights', 'Underwear']

pattern_lst = \
    ['pattern_All over pattern', 'pattern_Application/3D', 'pattern_Argyle',
       'pattern_Chambray', 'pattern_Check', 'pattern_Colour blocking',
       'pattern_Contrast', 'pattern_Denim', 'pattern_Dot',
       'pattern_Embroidery', 'pattern_Front print',
       'pattern_Glittering/Metallic', 'pattern_Hologram', 'pattern_Jacquard',
       'pattern_Lace', 'pattern_Melange', 'pattern_Mesh', 'pattern_Metallic',
       'pattern_Mixed solid/pattern', 'pattern_Neps',
       'pattern_Placement print', 'pattern_Sequin', 'pattern_Slub',
       'pattern_Solid', 'pattern_Stripe', 'pattern_Transparent',
       'pattern_Treatment']

index_lst = \
    ['Divided', 'Ladieswear', 'Menswear']

class_labels_dict = {'colour': colour_lst,
                     'product': product_lst,
                     'pattern': pattern_lst,
                     'index': index_lst}
                     
# Load model ResNet50 for embedding vectors calculation
resnet_model = tf.keras.applications.ResNet50(
    include_top=False, weights='imagenet', pooling='avg')

# Load the cloth items from the database with correpondent features an embedding vectors
df = pd.read_parquet('/content/drive/MyDrive/ds_project_dataset/embeddings_df.parquet')

def get_image_path_by_id(id):
    """
    Returns image_path correspondent to id (e.g. 220094021)
    """
    shared_folder_path = '/content/drive/MyDrive/ds_project_dataset/'
    id = str(id) 
    return shared_folder_path + 'images/' + '0'+id[:2]+'/0' + id + '.jpg'

94765736/94765736 [==============================] - 1s 0us/step


# Function

In [4]:
def get_recommendations(user_image):
  """
  Get recommendations similar/complementary based on the given user_image
  """
  # Resize the image to 80x80
  IMG_SIZE_FOR_RECOGNITION = 80
  user_image_for_recognition = cv2.resize(user_image, (IMG_SIZE_FOR_RECOGNITION, IMG_SIZE_FOR_RECOGNITION))
  
  # Get probabilities of the classes
  color_pred, type_pred, pattern_pred, index_pred = model.predict(np.array([user_image_for_recognition]))

  # Save the recognized features to the Dictionary
  pred_imag_dict = {
                    'product_group_name': class_labels_dict['product'][np.argmax(type_pred)],
                    'index_group_name': class_labels_dict['index'][np.argmax(index_pred)],
                    'perceived_colour_master_name': class_labels_dict['colour'][np.argmax(color_pred)]
                      }

  # Resize the image to 224x224
  IMG_SIZE_FOR_EMBEDDING = 224
  user_image_for_embedding = cv2.resize(user_image, (IMG_SIZE_FOR_EMBEDDING, IMG_SIZE_FOR_EMBEDDING))

  # Add an extra batch dimension to the image
  user_image_for_embedding = np.expand_dims(user_image_for_embedding, axis=0)

  # Pass the image through the ResNet50 model
  features = resnet_model.predict(user_image_for_embedding)

  # Extract the embedding from the output of the model
  embeddings = features.squeeze()

  # Add embedding to the dictionary
  pred_imag_dict['embedding'] = embeddings

  # Define database to look for similar items
  subset_similar = df[(df['product_group_name'] == pred_imag_dict['product_group_name']) & (df['index_group_name'] == pred_imag_dict['index_group_name'])].copy()

  # Define database to look for complementary items
  subset_comp = df[(df['product_group_name'] != pred_imag_dict['product_group_name']) & (df['index_group_name'] == pred_imag_dict['index_group_name'])].copy()

  #### Get recommendation of similar items
  # Caluclate similarity scores between user image and images from the database
  similarity_scores = []
  for array in subset_similar["embedding"]:
      array = array.reshape(1, -1)
      similarity = cosine_similarity(pred_imag_dict['embedding'].reshape(1, -1), array)
      similarity_scores.append(similarity[0][0])
  subset_similar['score'] = similarity_scores

  # Get TOP 10 similar products
  reccs = subset_similar.sort_values(by='score', ascending=False)[1:11]

  # Show results
  fig, ax = plt.subplots(2, 5, figsize=(3*3, 2*3))

  for i, axis in zip(range(10), ax.flatten()):
    img = cv2.imread(get_image_path_by_id(id=reccs['article_id'].iloc[i]))
    axis.imshow(img)
    axis.get_xaxis().set_visible(False)
    axis.get_yaxis().set_visible(False)
    #break
  #fig.suptitle("Recommendations of similar items", fontsize=26)
  plt.tight_layout();
  fig.savefig('static/recommendation_similar.jpg')

  #### Get recommendation of complementary items
  # Choose the same index_group_name. I.e. we recommend Ladieswear or Divided if the user has Ladieswear
  subset_index = \
    subset_comp[(subset_comp['index_group_name'] == pred_imag_dict['index_group_name']) | \
                (subset_comp['index_group_name'] == 'Divided')]

  # Choose the appropriate colour accordingly to the rule
  if pred_imag_dict['perceived_colour_master_name'] == 'Beige':
    color_to_recommend_lst = ['Blue', 'Purple', 'Brown', 'Mole', 'White', 'Black', 'Yellow', 'Orange']
  elif pred_imag_dict['perceived_colour_master_name'] == 'Black':
    color_to_recommend_lst = class_labels_dict['colour'].copy()
  elif pred_imag_dict['perceived_colour_master_name'] == 'Blue':
    color_to_recommend_lst = ['Red', 'Pink', 'Orange', 'Yellow', 'Grey', 'Metal', 'White', 'Black', 'Purple']
  elif pred_imag_dict['perceived_colour_master_name'] == 'Brown' or 'Mole':
    color_to_recommend_lst = ['Beige', 'Orange', 'White', 'Black']
  elif pred_imag_dict['perceived_colour_master_name'] == 'Green':
    color_to_recommend_lst = ['Orange', 'Purple', 'Grey','Metal', 'White', 'Black', 'Blue']
  elif pred_imag_dict['perceived_colour_master_name'] == 'Grey' or 'Metal':
    color_to_recommend_lst = ['Red', 'Pink', 'Blue', 'Purple', 'White', 'Black']
  elif pred_imag_dict['perceived_colour_master_name'] == 'Metal':
    color_to_recommend_lst = ['Red', 'Pink', 'Blue', 'Purple', 'White', 'Black']
  elif pred_imag_dict['perceived_colour_master_name'] == 'Orange':
    color_to_recommend_lst = ['Green', 'Blue', 'White', 'Black', 'Beige']
  elif pred_imag_dict['perceived_colour_master_name'] == 'Pink':
    color_to_recommend_lst = ['Blue', 'Grey', 'White', 'Black', 'Red', 'Beige']
  elif pred_imag_dict['perceived_colour_master_name'] == 'Purple':
    color_to_recommend_lst = ['Orange','Grey','Metal', 'Green', 'White', 'Black', 'Blue']
  elif pred_imag_dict['perceived_colour_master_name'] == 'Red':
    color_to_recommend_lst = ['Blue', 'Grey', 'White', 'Black', 'Pink', 'Beige']
  elif pred_imag_dict['perceived_colour_master_name'] == 'Turquoise':
    color_to_recommend_lst = ['Red', 'Pink', 'Orange', 'Yellow', 'Grey', 'Metal', 'White', 'Black', 'Purple']
  elif pred_imag_dict['perceived_colour_master_name'] == 'White':
    color_to_recommend_lst = class_labels_dict['colour'].copy()
  elif pred_imag_dict['perceived_colour_master_name'] == 'Yellow':
    color_to_recommend_lst = ['Green', 'Blue', 'White', 'Black', 'Beige']


  subset_index = subset_index[subset_index['perceived_colour_master_name'].isin(color_to_recommend_lst)]

  # Choose which type of clothes to recommmend
  types_to_recommend_lst = class_labels_dict['product'].copy()

  if pred_imag_dict['product_group_name'] == 'Garment Full body':
    types_to_recommend_lst.remove('Garment Full body')
    types_to_recommend_lst.remove('Garment Lower body')
    types_to_recommend_lst.remove('Garment Upper body')
  elif pred_imag_dict['product_group_name'] == 'Garment Lower body':
    types_to_recommend_lst.remove('Garment Full body')
    types_to_recommend_lst.remove('Garment Lower body')
  elif pred_imag_dict['product_group_name'] == 'Garment Upper body':
    types_to_recommend_lst.remove('Garment Full body')
    types_to_recommend_lst.remove('Garment Upper body')

  def get_most_similar_vectors(subset_comp, pred_imag_dict, ncols=3):
    # Caluclate similarity scores between user image and images from the database
    similarity_scores = []
    for array in subset_comp["embedding"]:
        array = array.reshape(1, -1)
        similarity = cosine_similarity(pred_imag_dict['embedding'].reshape(1, -1), array)
        similarity_scores.append(similarity[0][0])
    subset_comp['score'] = similarity_scores

    # Get TOP # similar products
    reccs = subset_comp.sort_values(by='score', ascending=False)[:ncols]
    return list(reccs['article_id'].values)

  id_product_lst = []
  ncols=3

  for type_to_recommend in types_to_recommend_lst:
    subset_group = subset_index[subset_index['product_group_name'] == type_to_recommend]
    subset_group = subset_group.copy()
    if subset_group.shape[0] == 0:
      continue
    id_product_lst.append(get_most_similar_vectors(subset_group, pred_imag_dict))
    #break

  # Flatten list of ID
  id_product_lst = [item for sublist in id_product_lst for item in sublist]

  # Show results
  nrows = int(len(id_product_lst)/3)
  fig, ax = plt.subplots(nrows, ncols, figsize=(3*ncols, 3*nrows))

  for index, axis in zip(id_product_lst, ax.flatten()):
    img = cv2.imread(get_image_path_by_id(id=index))
    axis.imshow(img)
    axis.get_xaxis().set_visible(False)
    axis.get_yaxis().set_visible(False)
    #break
  #fig.suptitle("Recommendations of complementary items", fontsize=26)
  plt.tight_layout();
  fig.savefig('static/recommendation_complementary.jpg')
  return

#Begining

# FLASK for Google Colab

In [5]:
!pip install flask --quiet
!pip install flask-ngrok --quiet
print("Completed!")

Completed!


In [6]:
# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup

!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

--2023-03-16 09:03:55--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 18.205.222.128, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13856790 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.tgz’

ngrok-stable-linux- 100%[===================>]  13.21M  40.1MB/s    in 0.3s    

2023-03-16 09:03:56 (40.1 MB/s) - ‘ngrok-stable-linux-amd64.tgz’ saved [13856790/13856790]



In [7]:
# extract the downloaded file using the following command 
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

ngrok


In [8]:
# paste your AuthToken here and execute this command
!./ngrok authtoken 

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
# import Flask from flask module
from flask import Flask, render_template, request

# import run_with_ngrok from flask_ngrok to run the app using ngrok
from flask_ngrok import run_with_ngrok
  
app = Flask(__name__) #app name
run_with_ngrok(app)
  
# @app.route("/")
# def hello():
#     return "Hello Friends! from Pykit.org. Thank you! for reading this article."

@app.route("/")
def home():
    return render_template("upload_page.html")

@app.route("/", methods=["POST"])
def upload():
    if request.method == "POST":
        file = request.files["file"]
        # Do something with the uploaded file here
        #filename = os.path.join(app.config["UPLOAD_FOLDER"], file.filename)
        file.save('static/user_image.jpg')
        
        # Read the uploaded image
        user_image = cv2.imread('static/user_image.jpg')
        # Get recommendations of similar/complementary items
        get_recommendations(user_image)        


        # create a string variable with the HTML code
        html = """
        <!DOCTYPE html>
        <html>
        <head>
          <title>Uploaded Images</title>
          <style>
            h1 {
        display: flex;
        align-items: flex-start;
        position: sticky;
        align-self: flex-end;
        color: #fffcf2;
        background-color: #403d39;
        padding: 1%;
        width:100%;
        height:10%;
      }
          h2 {
            color:#403d39;
            }

          body {
            background-attachment: scroll;
            background-color: #fffcf2;
            margin:0;
          }
          </style>
        </head>
        <body>
          <center><h1>OutfAIt</h1></center>
          <center><h2>User Image</h2>
          <img src="static/user_image.jpg" style="width:10%;height:10%;"><br>
          <h2>Recommendations of similar items</h2>
          <img src="static/recommendation_similar.jpg" style="width:50%;height:50%; alt='No similar items found'"><br>
          <h2>Recommendations of complementary items</h2>
          <img src="static/recommendation_complementary.jpg" style="width:50%;height:50%;" alt='No complementary items found'><br>
        </center></body>
        </html>
        """

        # create a new HTML file and write the HTML code to it
        with open("templates/uploaded_images.html", "w") as f:
            f.write(html)
        return render_template("uploaded_images.html")


if __name__ == "__main__":
    #app.config["UPLOAD_FOLDER"] = "/tmp"
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://23b0-34-72-205-24.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:32:45] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 203ms/step


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:33:08] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:33:08] "GET /static/user_image.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:33:08] "GET /static/recommendation_similar.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:33:08] "GET /static/recommendation_complementary.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:37:58] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 190ms/step


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:38:31] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:38:32] "GET /static/user_image.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:38:32] "GET /static/recommendation_complementary.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:38:32] "GET /static/recommendation_similar.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:38:39] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 187ms/step


INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:39:02] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:39:02] "GET /static/recommendation_complementary.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:39:02] "GET /static/user_image.jpg HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [16/Mar/2023 09:39:02] "GET /static/recommendation_similar.jpg HTTP/1.1" 200 -


In [ ]:
# !git branch
# !git add .
# !git commit -m "final notebook for recommendation update"
#!git push origin kuzma
